## Parámetros para la base de datos

In [ ]:
import numpy as np
import tifffile
import os

# Ruta de las imágenes
ruta_imagenes = "D:\BDnormalizada"

# Umbral para determinar si una imagen es "enferma"
umbral = 0.5  

# Recorrer todas las imágenes en la carpeta
for imagen_nombre in os.listdir(ruta_imagenes):
    # Leer la imagen
    imagen_tif = tifffile.imread(os.path.join(ruta_imagenes, imagen_nombre))

    # Normalizar la imagen
    imagen_normalizada = imagen_tif.astype(np.float32) / np.iinfo(imagen_tif.dtype).max
        
    # Determinar la etiqueta según la imagen normalizada
    if np.any(imagen_normalizada > umbral):
        print('-'*50)
        print(f'{imagen_nombre}: Cultivo enfermo')
        print(np.max(imagen_normalizada))
        print('-'*50)
    else:
        print('-'*50)
        print(f'{imagen_nombre}: Cultivo sano')
        print(np.max(imagen_normalizada))
        print('-'*50)
        

## Librerías

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from tkinter import Tk, Label, Button, Frame, filedialog, Entry
from PIL import ImageTk, Image
import numpy as np

## Creación de la base de datos

In [ ]:
# Directorio de la carpeta imágenes
dir = "D:\BDnormalizada"
Catg = ['CULTIVO ENFERMO', 'CULTIVO SANO']
data = []

for category in Catg:
    path = os.path.join(dir, category)
    label = Catg.index(category)
    
    for img in os.listdir(path):
        imgpath = os.path.join(path, img)
        Br_img = cv2.imread(imgpath, 0)
        try:
            Br_img = cv2.resize(Br_img, (50, 50))
            image = np.array(Br_img).flatten()
            imagen_normalizada = image.astype(np.float32) / np.iinfo(image.dtype).max
            data.append([imagen_normalizada, label])
        except Exception as e:
            pass

print(len(data))

pick_in = open('BDnormalizada.pickle', 'wb')
pickle.dump(data, pick_in)
pick_in.close()

## Entrenamiento del modelo

In [ ]:
pick_in = open('BDnormalizada.pickle', 'rb')
data = pickle.load(pick_in)
pick_in.close()

data_ab = data

random.shuffle(data)
features = []
labels = []

for feature, label in data:
    features.append(feature)
    labels.append(label)

Catg = ['CULTIVO ENFERMO', 'CULTIVO SANO']


xtrain, xtest, ytrain, ytest = train_test_split(
    features, labels, test_size=0.7)

model = SVC(C=100, kernel='poly', gamma=1)
model.fit(xtrain, ytrain)

pick = open('modelo.sav', 'wb')
pickle.dump(model, pick)
pick.close()

## Predicciones usando la bd de entrenamiento

In [ ]:
pick = open('modelo.sav', 'rb')
model = pickle.load(pick)
pick.close()

prediction = model.predict(xtest)
accuracy = model.score(xtest, ytest)

categories = ['CULTIVO ENFERMO', 'CULTIVO SANO']

print('Accuracy: ', accuracy)
print('prediction is: ', categories[prediction[0]])

mypet = xtest[0].reshape(50,50)
plt.imshow(mypet, cmap='gray')
plt.show()

## Predicciones con una nueva imagen

In [ ]:
# Cargar la nueva imagen
new_img_path = "D:\IMAGENES\DJI_20230606111754_0001_MS_NIR.TIF"

# Cargar en escala de grises
flattened_img = cv2.imread(new_img_path, 0) 

# Procesar la imagen
try:
    resized_img = cv2.resize(flattened_img, (50, 50))
    flattened_img = np.array(resized_img).flatten()
    imagen_normalizada = flattened_img.astype(np.float32) / np.iinfo(flattened_img.dtype).max
except Exception as e:
    pass

# Cargar el modelo SVM
pick = open('modelo.sav', 'rb')
model = pickle.load(pick)
pick.close()

# Predecir la categoría de la nueva imagen
prediction = model.predict([imagen_normalizada])


# Mostrar la predicción
categories = ['CULTIVO ENFERMO', 'CULTIVO SANO']
print('La nueva imagen pertenece a la categoría:', categories[prediction[0]])

# Mostrar la nueva imagen
plt.imshow(resized_img, cmap='gray')
plt.show()